In [3]:
# include modules relative to this file's directory
include(joinpath(@__DIR__, "../src/utils.jl"))
include(joinpath(@__DIR__, "../src/model.jl"))
include(joinpath(@__DIR__, "../src/analysis.jl"))

using .Utils
using .Model
using .Analysis
using Plots, LaTeXStrings

# make Figures directory if missing
isdir(joinpath(@__DIR__, "..", "figures")) || mkpath(joinpath(@__DIR__, "..", "figures"))

# ==== parameters ====
k = 1.0
K = 15.0
ξ = 0.1
ωn = 8.0
θ_ref = 0.5
dt = 1e-3
c = 0.2

# search ranges
β_range = range(0.01, stop=0.5, length=500)
w_range = range(0.1, stop=12, length=500)

# find best (w, beta) using Analysis
w_best, β_best, _ = Analysis.find_best_beta_w(β_range, w_range; K=K, ωn=ωn, ξ=ξ, k=k, θ_ref=θ_ref)
println("Found w_best=", w_best, " β_best=", β_best)

# compute γ_ref / γ_opt
β_ref = β_best
ω_ref = w_best

γ = (β_ref) * exp(c*(pi/(2*ω_ref)))*(1 - exp(-c*(pi/ω_ref)))

# run one simulation (use γ_opt)
sol, logs = run_simulation(dt=dt, Tfinal=50.0, ξ=ξ, ωn=ωn, c=c, γ=γ, β_ref=β_ref, k=k, K=K, θ_ref=θ_ref)

# run one simulation (use γ_opt ; 0.95)
sol1, logs1 = run_simulation(dt=dt, Tfinal=50.0, ξ=ξ, ωn=ωn, c=c, γ=0.95*γ, β_ref=β_ref, k=k, K=K, θ_ref=θ_ref)

# plots
s = scatter(logs.ti_arr, logs.Ti_arr, 
    markersize=2,
    label=L"\beta:\gamma^\star = (e^{c\frac{\pi}{2\omega^\star}} - e^{-c\frac{\pi}{2\omega^\star}}) \beta^\star",
    xlabel=L"t", 
    ylabel=L"\beta(t)",
    legendfontsize=9,
    xlabelfontsize=9,  
    ylabelfontsize=9,
    tickfontsize=8,
    # size=(504, 288)
    size=(504, 184)
)
scatter!(logs1.ti_arr, logs1.Ti_arr, markersize=2, color="red", label=L"\beta:\gamma = 0.95 \cdot \gamma^\star")
hline!([β_ref], linestyle=:dash, label=L"\beta=\beta^\star")

# save the figure into project-level /figures
outpath = joinpath(@__DIR__, "..", "figures", "fig4.pdf")
savefig(s, outpath)
println("Saved ", outpath)


Found w_best=7.707414829659319

 β_best=0.09150300601202405
Saved /Users/taya/Documents/neuromorphicControl/pendulum/simulation/fig-repro/figures/../figures/fig4.pdf
